In [1]:
import telegram, logging, subprocess, os, apiwrapper

from telegram.ext import Updater, CommandHandler, Filters, MessageHandler

from chatbotwrapper import PythonChatBot
from loggers import get_logger

In [2]:
chatbot = PythonChatBot(threshold=0.5)

In [3]:
train_log = get_logger('train_bot.log', logging.INFO)
test_log = get_logger('test_bot.log', logging.INFO)

In [4]:
def post_user_location(bot, update):
    reply_markup = telegram.ReplyKeyboardMarkup([[telegram.KeyboardButton('Поделиться местоположением', request_location=True)]])
    bot.sendMessage(chat_id=update.message.chat_id, text='Я найду ближайший банкомат, но мне нужно знать ваше местоположение ', reply_markup=reply_markup)

In [5]:
def get_user_location(bot, update):
    lat = update.message.location.latitude
    lon = update.message.location.longitude
    reply_markup = telegram.ReplyKeyboardRemove()
    bot.send_message(chat_id=update.message.chat_id, text="Он находится вот здесь", reply_markup=reply_markup)
    answer = apiwrapper.get_location(lat, lon)
    if type(answer) == 'str':
        bot.send_message(chat_id=update.message.chat_id, text="Что-то пошло не так. Попробуйте снова")
    else:
        bot.send_location(update.message.chat_id, answer[0], answer[1], text='Ближайший банкомат')

In [6]:
def get_currency(bot, update):
    rates = apiwrapper.get_rates()
    if type(rates) == 'str':
        bot.send_message(chat_id=update.message.chat_id, text=rates)
        return
    selling = "Вот по таким ценам сейчас проходит продажа иностранных валют\n" + '\n'.join(['{} -> {} рублей за штуку'.format(i['name'], i['value']) for i in rates[0]])
    bot.send_message(chat_id=update.message.chat_id, text=selling)
    buying = "А вот по таким ценам проходит покупка иностранных валют\n" + '\n'.join(['{} -> {} рублей за штуку'.format(i['name'], i['value']) for i in rates[1]])
    bot.send_message(chat_id=update.message.chat_id, text=buying)


In [7]:
bot_train = telegram.Bot(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

updater_train = Updater(token='386950793:AAFWQO5Exopgk-WwzHcrKEArN7tVM20OL4A')

dispatcher_train = updater_train.dispatcher

In [8]:
def start_train(bot, update):    
    bot.send_message(chat_id=update.message.chat_id, text="""Привет! Я готов обучаться!\nПришли сообщением вопрос и ответ на него.\nНе забудь разделить их вот этим симоволом "|" """)

In [9]:
def train(bot, update):
    user = update.message.from_user
    text = update.message.text
    train_log.info("text of %s: %s" % (user.first_name, text))
    bot.send_message(chat_id=update.message.chat_id, text="""Спасибо, я постараюсь хорошо обучиться.\nПокидай обучалок еще, мне нравится учиться""")
    try:
        train_text = text.split('|')
        chatbot.train(train_text[0], train_text[1])
    except Exception as e:
        train_log.warning("Произошла ошибка -{}- \nНа обучающем тексте -{}-".format(e, text))
        bot.send_message(chat_id=update.message.chat_id, text="Не получилось обучиться. Может быть неправильный формат?")
    else:
        train_log.info("Обучение на обучающем тексте -{}- прошло успешно".format(text))

In [10]:
start_handler_train = CommandHandler('start', start_train)
text_handler_train = MessageHandler(Filters.text & (~Filters.location), train)

dispatcher_train.add_handler(start_handler_train)
dispatcher_train.add_handler(text_handler_train)

In [11]:
updater_train.start_polling()

In [12]:
bot_predict = telegram.Bot(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

updater_predict = Updater(token='383933408:AAHmUeE2FX_tODZfiqZUSDMNHYuO-31q1p4')

dispatcher_predict = updater_predict.dispatcher

chatbot_predict = PythonChatBot(threshold=0.5, read_only=True)

In [13]:
def start_predict(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text="""Привет! Меня еще учат, но кажется что я более-менее готов отвечать на вопросы""")

In [14]:
def predict(bot, update):
    user = update.message.from_user
    text = update.message.text
    test_log.info("text of %s: %s" % (user.first_name, text))
    try:
        response = str(chatbot_predict.response(text))
    except Exception as e:
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")
        test_log.warning("Произошла ошибка -{}- \nПытался ответить на -{}-".format(e, text))
    else:
        bot.send_message(chat_id=update.message.chat_id, text=response)
        if response.strip().lower() == 'Найти ближайший банкомат?'.strip().lower():
            post_user_location(bot, update)
        elif response.strip().lower() == 'Курсы валют?'.strip().lower():
            get_currency(bot, update)
        test_log.info("Ответ на вопрос -{}- прошел успешно".format(text))

In [15]:
def predict_sound(bot, update):
    user = update.message.from_user
    filepath = '{}_question'.format(user.first_name)
    try:
        file_id = update.message.voice.file_id
        newFile = bot.get_file(file_id)
        newFile.download(filepath + '_wb.mp3')
        text = apiwrapper.sound2text(filepath)
        response = str(chatbot_predict.response(text))
    except Exception as e:
        test_log.warning("Произошла ошибка -{}- \nПытался ответить на звук".format(e))
        bot.send_message(chat_id=update.message.chat_id, text="К сожалению что-то пошло не так")
    else:
        bot.send_message(chat_id=update.message.chat_id, text=response)
        if response.strip().lower() == 'Найти ближайший банкомат?'.strip().lower():
            post_user_location(bot, update)
        elif response.strip().lower() == 'Курсы валют?'.strip().lower():
            get_currency(bot, update)
        test_log.info("Ответ на вопрос -{}- прошел успешно. Ответ на звук".format(str(text)))

In [16]:
start_handler_predict = CommandHandler('start', start_predict)
text_handler_predict = MessageHandler(Filters.text & (~Filters.voice) & (~Filters.location), predict)
voice_handler_predict = MessageHandler(Filters.voice & (~Filters.location), predict_sound)
location_handler = MessageHandler(Filters.location, get_user_location)

dispatcher_predict.add_handler(start_handler_predict)
dispatcher_predict.add_handler(text_handler_predict)
dispatcher_predict.add_handler(voice_handler_predict)
dispatcher_predict.add_handler(location_handler)


In [17]:
updater_predict.start_polling()